In [1]:
# here, we combine all .csv files into the single file total_state_data.csv
# all .csv files to be combined should be placed in ./Data directory

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df1 = pd.read_csv('../../SharedData/dataset-generation-final/population-1990-2020-final.csv')
df2 = pd.read_csv('../../SharedData/dataset-generation-final/monthly_emissions_1990_2024')
df3 = pd.read_csv('../../SharedData/dataset-generation-final/monthly-weather-1990-2019-final.csv')
df4 = pd.read_csv('../../SharedData/dataset-generation-final/gdp-1997-2023-final.csv')
df5 = pd.read_csv('../../SharedData/dataset-generation-final/energy_final.csv')
df6 = pd.read_csv('../../SharedData/dataset-generation-final/power_plant_count.csv')

In [2]:
df6.head()

,Unnamed: 0,year,month,state,monthly_num_plants,date,yearly_num_plants
0,0,1997,1,AL,11,1997-01,10
1,1,1997,1,AR,7,1997-01,6
2,2,1997,1,AZ,7,1997-01,9
3,3,1997,1,CA,24,1997-01,24
4,4,1997,1,CO,14,1997-01,14


In [3]:
# remove duplicate indexing columns

df1 = df1.drop('Unnamed: 0', axis=1)
df4 = df4.drop('Unnamed: 0', axis=1)
df6 = df6.drop('Unnamed: 0', axis=1)

# remove duplicate year/month columns

df6 = df6.drop(['year','month'], axis=1)

# fix datetime formatting

df2['date'] = df2['date'].apply(pd.to_datetime)
df2['date'] = df2['date'].dt.strftime('%Y-%m')

In [4]:
total_df = df1.merge(df2, on=['state','date'], how='right').merge(df3, on=['state','date'], how='left').merge(df4, on=['state','date'], how='left').merge(df5, on=['state','date'], how='left').merge(df6, on=['state','date'], how='left')

In [5]:
total_df['date']=pd.to_datetime(total_df['date'])

In [6]:
total_df.to_csv('../../SharedData/total_state_data.csv',index=False)